In [1]:
import pandas as pd
import os

dataset_rename = {'cw': 'ContentWise', 'rl4rs': 'RL4RS'}
metrics_rename = {'f1':'F1', 'accuracy': 'Accuracy', 'roc-auc':"ROC-AUC"}
embeddings_rename = {'svd':'SVD', 'neural':"NN", 'explicit':"Ext."}

files = [file for file in os.listdir('.') if file.endswith('csv') and file.split('_')[0] in dataset_rename]
model_names = list(set([file.split('_')[-1].split('.')[0] for file in files]))
print('== all models ==', *model_names, sep='\n')

# slatewise
slatewise_models = {
    'NeuralClickModel':'NCM',
    'NeuralClickModelDiffSample':'NCM-softmax',
    'MatrixFactorization': 'MF', 
    'LogReg': 'Logistic regression', 
    'TRUESlatewiseGRU': 'Slate-wise GRU', 
    'AGGSlatewiseGRU':'Slate-wise GRU + Aggregation', 
    'SlatewiseAttention' : 'Slate-wise Transformer',
    'SlatewiseAttentionAnorherSeed' : 'SlatewiseAttentionAnorherSeed'
}



# sessionwise
sessionwise_models = {
    
    'SCOT': 'SCOT', 
    'AttentionGRU-FIXED' : 'Transformer + GRU', 
    # 'AttentionGRU' : 'Transformer + GRU broken', 
    'SessionwiseGRU': 'OLD Session-wise GRU ',
    'TRUESessionwiseGRU':'Session-wise GRU',
    'SessionwiseAttention': 'Session-wise Transformer'
    
}

model_names = slatewise_models
model_names.update(sessionwise_models)

== all models ==
AGGSlatewiseGRU
SCOT
SlatewiseAttentionAnorherSeed
MatrixFactorization
TRUESlatewiseGRU
NeuralClickModelDiffSample
TRUESessionwiseGRU
SlatewiseAttention
SlatewiseGRUwithAggregation
SessionwiseGRU
SessionwiseAttention
LogReg
AttentionGRU-FIXED
NeuralClickModel


In [2]:
data = {}

index = pd.MultiIndex.from_product(
    [["ContentWise", "RL4RS"], ["ROC-AUC", "F1", "Accuracy"]]
)

columns = pd.MultiIndex.from_product(
    [model_names.values(), ["SVD", "NN", "Ext."]]
)


df = pd.DataFrame(
    index=index,
    columns=columns,
)
data = df.to_dict()
data

{('NCM', 'SVD'): {('ContentWise', 'ROC-AUC'): nan,
  ('ContentWise', 'F1'): nan,
  ('ContentWise', 'Accuracy'): nan,
  ('RL4RS', 'ROC-AUC'): nan,
  ('RL4RS', 'F1'): nan,
  ('RL4RS', 'Accuracy'): nan},
 ('NCM', 'NN'): {('ContentWise', 'ROC-AUC'): nan,
  ('ContentWise', 'F1'): nan,
  ('ContentWise', 'Accuracy'): nan,
  ('RL4RS', 'ROC-AUC'): nan,
  ('RL4RS', 'F1'): nan,
  ('RL4RS', 'Accuracy'): nan},
 ('NCM', 'Ext.'): {('ContentWise', 'ROC-AUC'): nan,
  ('ContentWise', 'F1'): nan,
  ('ContentWise', 'Accuracy'): nan,
  ('RL4RS', 'ROC-AUC'): nan,
  ('RL4RS', 'F1'): nan,
  ('RL4RS', 'Accuracy'): nan},
 ('NCM-softmax', 'SVD'): {('ContentWise', 'ROC-AUC'): nan,
  ('ContentWise', 'F1'): nan,
  ('ContentWise', 'Accuracy'): nan,
  ('RL4RS', 'ROC-AUC'): nan,
  ('RL4RS', 'F1'): nan,
  ('RL4RS', 'Accuracy'): nan},
 ('NCM-softmax', 'NN'): {('ContentWise', 'ROC-AUC'): nan,
  ('ContentWise', 'F1'): nan,
  ('ContentWise', 'Accuracy'): nan,
  ('RL4RS', 'ROC-AUC'): nan,
  ('RL4RS', 'F1'): nan,
  ('RL4RS',

In [3]:
for file in files:
    dataset = dataset_rename[file.split('_')[0]]
    try:
        model = model_names[file.split('_')[-1].split('.')[0]]
    except:
        print(f'Warning: {file} is unknown model')
        continue
    data_dict = pd.read_csv(file).drop('Unnamed: 0', axis=1).set_index('embeddings').to_dict('index')
    for embedding, m in data_dict.items():
        for metric, score in m.items():
#             print(model, embeddings_rename[embedding], dataset, metrics_rename[metric], score )
            try:
                data[model, embeddings_rename[embedding]][dataset, metrics_rename[metric]] = score
            except:
                pass
        

In [4]:
df = pd.DataFrame(
    data,
    index=index,
    columns=columns,
).T.dropna(how='all')
df

ContentWise                         RL4RS  \
                                       ROC-AUC        F1  Accuracy   ROC-AUC   
NCM                           SVD     0.672457  0.249699  0.890091  0.914036   
                              NN      0.719708  0.297595  0.860661  0.923075   
                              Ext.         NaN       NaN       NaN  0.918838   
NCM-softmax                   SVD     0.671761  0.257408  0.889826  0.908454   
                              NN      0.729533  0.283293  0.891382  0.918583   
                              Ext.         NaN       NaN       NaN  0.915784   
MF                            SVD     0.652690  0.197764  0.260027  0.716958   
Logistic regression           SVD     0.626372  0.240592  0.769028  0.868028   
                              NN      0.684548  0.274763  0.758625  0.911916   
                              Ext.         NaN       NaN       NaN  0.902556   
Slate-wise GRU                SVD     0.679536  0.279646  0.803955  0.901342   
                              NN      0.743958  0.331039  0.860347  0.923913   
                              Ext.         NaN       NaN       NaN  0.923439   
Slate-wise GRU + Aggregation  SVD     0.656881  0.230317  0.869834  0.859963   
                              NN      0.724710  0.300113  0.825600  0.851119   
                              Ext.         NaN       NaN       NaN  0.855602   
Slate-wise Transformer        SVD     0.668577  0.274205  0.886928  0.899657   
                              NN      0.740148  0.328198  0.838747  0.929887   
                              Ext.         NaN       NaN       NaN  0.924882   
SlatewiseAttentionAnorherSeed SVD     0.672834  0.282949  0.871526  0.895843   
                              NN      0.733017  0.318922  0.849380  0.927598   
                              Ext.         NaN       NaN       NaN  0.924200   
SCOT                          SVD     0.671757  0.297491  0.863569  0.909597   
                              NN      0.715095  0.306451  0.812614  0.933914   
                              Ext.         NaN       NaN       NaN  0.927273   
Transformer + GRU             SVD     0.625189  0.192788  0.248216  0.858889   
                              NN      0.737329  0.317201  0.849732  0.876702   
                              Ext.         NaN       NaN       NaN  0.871765   
OLD Session-wise GRU          SVD     0.665810  0.255503  0.861826  0.917065   
                              NN      0.731758  0.304215  0.757926  0.921791   
                              Ext.         NaN       NaN       NaN  0.919225   
Session-wise GRU              SVD     0.678468  0.261236  0.859478  0.863005   
                              NN      0.740156  0.308166  0.738505  0.859461   
                              Ext.         NaN       NaN       NaN  0.860901   
Session-wise Transformer      SVD     0.637128  0.261067  0.817086  0.880952   
                              NN      0.703817  0.284120  0.773755  0.925115   
                              Ext.         NaN       NaN       NaN  0.913644   

                                                        
                                          F1  Accuracy  
NCM                           SVD   0.879844  0.846306  
                              NN    0.887764  0.854189  
                              Ext.  0.884437  0.851723  
NCM-softmax                   SVD   0.867572  0.835256  
                              NN    0.883655  0.847830  
                              Ext.  0.881854  0.847854  
MF                            SVD   0.761080  0.659001  
Logistic regression           SVD   0.850969  0.790521  
                              NN    0.879420  0.836658  
                              Ext.  0.871683  0.820070  
Slate-wise GRU                SVD   0.867143  0.825608  
                              NN    0.878035  0.832136  
                              Ext.  0.883845  0.845315  
Slate-wise GRU + Aggregation  SVD   0.851310  0.795865  
  

In [5]:
df = df.style.format(decimal='.', precision=3).highlight_max(axis=0, props="font-weight:bold;")

In [6]:
df

In [7]:
print(df.to_latex(convert_css=True))

\begin{tabular}{llrrrrrr}
 &  & \multicolumn{3}{r}{ContentWise} & \multicolumn{3}{r}{RL4RS} \\
 &  & ROC-AUC & F1 & Accuracy & ROC-AUC & F1 & Accuracy \\
\multirow[c]{3}{*}{NCM} & SVD & 0.672 & 0.250 & 0.890 & 0.914 & 0.880 & 0.846 \\
 & NN & 0.720 & 0.298 & 0.861 & 0.923 & 0.888 & 0.854 \\
 & Ext. & nan & nan & nan & 0.919 & 0.884 & 0.852 \\
\multirow[c]{3}{*}{NCM-softmax} & SVD & 0.672 & 0.257 & 0.890 & 0.908 & 0.868 & 0.835 \\
 & NN & 0.730 & 0.283 & \bfseries 0.891 & 0.919 & 0.884 & 0.848 \\
 & Ext. & nan & nan & nan & 0.916 & 0.882 & 0.848 \\
MF & SVD & 0.653 & 0.198 & 0.260 & 0.717 & 0.761 & 0.659 \\
\multirow[c]{3}{*}{Logistic regression} & SVD & 0.626 & 0.241 & 0.769 & 0.868 & 0.851 & 0.791 \\
 & NN & 0.685 & 0.275 & 0.759 & 0.912 & 0.879 & 0.837 \\
 & Ext. & nan & nan & nan & 0.903 & 0.872 & 0.820 \\
\multirow[c]{3}{*}{Slate-wise GRU} & SVD & 0.680 & 0.280 & 0.804 & 0.901 & 0.867 & 0.826 \\
 & NN & \bfseries 0.744 & \bfseries 0.331 & 0.860 & 0.924 & 0.878 & 0.832 \\
 & Ext. & 

n{tabular}{llrrrrrr}
 &  & \multicolumn{3}{r}{ContentWise} & \multicolumn{3}{r}{RL4RS} \\
 &  & ROC-AUC & F1 & Accuracy & ROC-AUC & F1 & Accuracy \\
MF & SVD & 0.653 & 0.198 & 0.260 & 0.717 & 0.761 & 0.659 \\
\multirow[c]{2}{*}{Logistic regression} & SVD & 0.626 & 0.241 & 0.769 & nan & nan & nan \\
 & nn.Embeddings & 0.685 & 0.275 & 0.759 & nan & nan & nan \\
\multirow[c]{3}{*}{Slate-wise Transformer} & SVD & 0.663 & 0.281 & 0.836 & 0.900 & 0.866 & 0.820 \\
 & nn.Embeddings & 0.733 & \bfseries 0.325 & 0.835 & 0.927 & 0.883 & 0.844 \\
 & Explicit & nan & nan & nan & 0.922 & 0.883 & 0.846 \\
\multirow[c]{3}{*}{SCOT} & SVD & 0.672 & 0.297 & \bfseries 0.864 & 0.910 & 0.875 & 0.831 \\
 & nn.Embeddings & 0.715 & 0.306 & 0.813 & \bfseries 0.934 & \bfseries 0.896 & \bfseries 0.865 \\
 & Explicit & nan & nan & nan & 0.927 & 0.891 & 0.860 \\
\multirow[c]{3}{*}{Transformer + GRU} & SVD & 0.625 & 0.193 & 0.248 & 0.859 & 0.842 & 0.791 \\
 & nn.Embeddings & \bfseries 0.737 & 0.317 & 0.850 & 0.877 & 0.856 & 0.808 \\
 & Explicit & nan & nan & nan & 0.872 & 0.854 & 0.804 \\
\multirow[c]{3}{*}{Session-wise GRU} & SVD & 0.666 & 0.256 & 0.862 & 0.917 & 0.882 & 0.848 \\
 & nn.Embeddings & 0.732 & 0.304 & 0.758 & 0.922 & 0.886 & 0.851 \\
 & Explicit & nan & nan & nan & 0.919 & 0.884 & 0.850 \\
\multirow[c]{3}{*}{Session-wise Transformer} & SVD & 0.637 & 0.261 & 0.817 & 0.881 & 0.849 & 0.782 \\
 & nn.Embeddings & 0.704 & 0.284 & 0.774 & 0.925 & 0.885 & 0.843 \\
 & Explicit & nan & nan & nan & 0.914 & 0.876 & 0.831 \\
\end{tabular}

